In [ ]:
using  GridapGmsh
using  Gridap
using  Gridap.Geometry
using  Gridap.TensorValues
using  Plots
using LinearAlgebra
using LineSearches: BackTracking
using Gridap.Arrays
using Gridap.ReferenceFEs

In [ ]:
const  E_mat = 210e3
const  ν_mat = 0.3
const  Gc = 2.7
const  ls = 0.0075
const  η = 1e-15

In [ ]:
path = "./fatigueresults$ls/"                                # Output path
mkpath(path) 

In [ ]:
cd("fatigueresults$ls")

In [ ]:
using  Gmsh: gmsh
const  L = 0.3
const  H = 1.0
const  CL = 0.06 #Crack  height
const  CH = H/100 #Crack  height
const  CW = 0.05
const  hfc = 0.005/100 #Mesh  size  parameter
const  hf = 0.005/4 #Mesh  size  parameter
const  h = 25*hf #Mesh  size  parameter
const  FMR = 4*0.010
gmsh.initialize()
gmsh.option.setNumber("General.Terminal", 1)
gmsh.model.geo.addPoint(0.0, 0.0 , 0.0, h ,1)
gmsh.model.geo.addPoint(L, 0.0, 0.0, h, 2)
gmsh.model.geo.addPoint(L, H/2, 0.0, h, 3)
gmsh.model.geo.addPoint(L, H/2 + CW/2 - CH/2, 0.0, h, 4)
gmsh.model.geo.addPoint(L-(CL-CH), H/2 + CW/2 - CH/2, 0.0, h, 5)
gmsh.model.geo.addPoint(L-(CL), H/2 + CW/2, 0.0, h, 6)
gmsh.model.geo.addPoint(L-(CL-CH), H/2 + CW/2 + CH/2, 0.0, h, 7)
gmsh.model.geo.addPoint(L, H/2 + CW/2 + CH/2, 0.0, h, 8)
gmsh.model.geo.addPoint(L, H, 0.0, h, 9)
gmsh.model.geo.addPoint(0.0, H, 0.0, h, 10)
gmsh.model.geo.addPoint(0.0, H/2, 0.0, h, 11)
gmsh.model.geo.addPoint(0.0, H/2 - CW/2 + CH/2, 0.0, h, 12)
gmsh.model.geo.addPoint(CL-CH, H/2 - CW/2 + CH/2, 0.0, h, 13)
gmsh.model.geo.addPoint(CL, H/2 - CW/2, 0.0, h, 14)
gmsh.model.geo.addPoint(CL-CH, H/2 - CW/2 - CH/2, 0.0, h, 15)
gmsh.model.geo.addPoint(0, H/2 - CW/2 - CH/2, 0.0, h, 16)

gmsh.model.geo.addLine(1, 2, 1)
gmsh.model.geo.addLine(2, 3, 2)
gmsh.model.geo.addLine(3, 4, 3)
gmsh.model.geo.addLine(4, 5, 4)
gmsh.model.geo.addLine(5, 6, 5)
gmsh.model.geo.addLine(6, 7, 6)
gmsh.model.geo.addLine(7, 8, 7)
gmsh.model.geo.addLine(8, 9, 8)
gmsh.model.geo.addLine(9, 10, 9)
gmsh.model.geo.addLine(10, 11, 10)
gmsh.model.geo.addLine(11, 12, 11)
gmsh.model.geo.addLine(12, 13, 12)
gmsh.model.geo.addLine(13, 14, 13)
gmsh.model.geo.addLine(14, 15, 14)
gmsh.model.geo.addLine(15, 16, 15)
gmsh.model.geo.addLine(16, 1, 16)

gmsh.model.geo.addCurveLoop([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16],1)

gmsh.model.geo.addPlaneSurface([1], 1)
gmsh.model.addPhysicalGroup(2, [1],1)

gmsh.model.addPhysicalGroup(0, [3],1)
gmsh.model.addPhysicalGroup(0, [11],2)

gmsh.model.addPhysicalGroup(1, [1],1)
gmsh.model.addPhysicalGroup(1, [9],2)


gmsh.model.setPhysicalName(2, 1, "Domain")

gmsh.model.setPhysicalName(0, 1, "DirichletRightPoint")
gmsh.model.setPhysicalName(0, 2, "DirichletLeftPoint")

gmsh.model.setPhysicalName(1, 1, "DirichletBottom")
gmsh.model.setPhysicalName(1, 2, "DirichletTop")

gmsh.model.mesh.field.add("Box", 10)
gmsh.model.mesh.field.setNumber(10, "VIn", hf)
gmsh.model.mesh.field.setNumber(10, "VOut", h)
gmsh.model.mesh.field.setNumber(10, "XMin", 0)
gmsh.model.mesh.field.setNumber(10, "XMax", L)
gmsh.model.mesh.field.setNumber(10, "YMin", 0.5*H - 2*FMR)
gmsh.model.mesh.field.setNumber(10, "YMax", 0.5*H + 2*FMR)

gmsh.model.mesh.field.setAsBackgroundMesh(10)
gmsh.model.geo.synchronize()
gmsh.model.mesh.generate(2)
gmsh.write("SENT_Mesh.msh")
gmsh.finalize()

In [ ]:
model = GmshDiscreteModel("SENT_Mesh.msh")
writevtk(model ,"SENT_Mesh") 

In [ ]:
const σc = 2445.42 
const m = 3*E_mat*Gc/(4*ls*σc^2)
const ψ_Crit = σc^2 / (2*E_mat)
const kf = 0.5
const αT = ψ_Crit/kf

In [ ]:
function degDer(ϕ)
    g = (m*ϕ)/(0.25*m^2*ϕ^4 - 0.5*m^2*ϕ^2 + 0.25*m^2 - m*ϕ^4 + m*ϕ^2 + ϕ^4) 
    return g
end

In [ ]:
function  ElasFourthOrderConstTensor(E,ν,PlanarState)
    # 1 for  Plane  Stress  and 2 Plane  Strain  Condition
    if  PlanarState  == 1
        C1111 =E/(1-ν*ν)
        C1122 = (ν*E)/(1-ν*ν)
        C1112 = 0.0
        C2222 =E/(1-ν*ν)
        C2212 = 0.0
        C1212 =E/(2*(1+ν))
    elseif  PlanarState  == 2
        C1111 = (E*(1-ν*ν))/((1+ν)*(1-ν-2*ν*ν))
        C1122 = (ν*E)/(1-ν-2*ν*ν)
        C1112 = 0.0
        C2222 = (E*(1-ν))/(1-ν-2*ν*ν)
        C2212 = 0.0
        C1212 =E/(2*(1+ν))
    end
    C_ten = SymFourthOrderTensorValue(C1111 ,C1112 ,C1122 ,C1112 ,C1212 ,C2212 ,C1122 ,C2212 ,C2222)
    return   C_ten
end
     

const  C_mat = ElasFourthOrderConstTensor(E_mat ,ν_mat ,2)
     

In [ ]:
I2 = SymTensorValue{2,Float64}(1.0 ,0.0 ,1.0)
I4 = I2⊗I2
I4_sym = one(SymFourthOrderTensorValue{2,Float64})
P_vol = (1.0/2)*I4
P_dev = I4_sym  - P_vol
     

In [ ]:
function σfun(ε,ε_in,s_in)
    σ_elas = C_mat⊙ε
    gs = (s_in^2) / (s_in^2 + m*(1-s_in)*(0.5+0.5*s_in))
    if tr(ε_in) >= 0
        σ = (gs)*σ_elas
    elseif  tr(ε_in) < 0
        σ = (gs)*P_dev ⊙ σ_elas + P_vol ⊙ σ_elas
    end
    return  σ
end
     

In [ ]:
function ψPos(ε_in,Fdg)
   σ_elas = C_mat⊙ε_in
    if tr(ε_in) >= 0
        ψPlus = 0.5*(ε_in ⊙ σ_elas)/(Fdg)
    elseif  tr(ε_in) < 0
        ψPlus = 0.5*((P_dev ⊙ σ_elas)⊙(P_dev⊙ε_in))/(Fdg)
    end
    return ψPlus
end

function ψPosOrg(ε_in)
   σ_elas = C_mat⊙ε_in
    if tr(ε_in) >= 0
        ψPlus = 0.5*(ε_in ⊙ σ_elas)
    elseif  tr(ε_in) < 0
        ψPlus = 0.5*((P_dev ⊙ σ_elas)⊙(P_dev⊙ε_in))
    end
    return ψPlus
end

In [ ]:
function  new_EnergyState(ψPlusPrev_in,ψhPos_in)
    ψPlus_in = ψhPos_in
    if ψPlus_in  >=ψPlusPrev_in
        ψPlus_out =ψPlus_in
    else
        ψPlus_out=ψPlusPrev_in
    end
    true,ψPlus_out
end
     

In [ ]:
function  project(q,model ,dΩ,order)
    reffe = ReferenceFE(lagrangian ,Float64 ,order)
    V = FESpace(model ,reffe ,conformity =:L2)
    a(u,v) =∫(u*v)*dΩ
    b(v) =∫(v*q)*dΩ
    op = AffineFEOperator(a,b,V,V)
    qh = solve(op)
    return  qh
end
     

In [ ]:
function FatigueDegrad(αb)
    fdg = @. ifelse(αb >= αT , ((2*αT)/(αb + αT))*((2*αT)/(αb + αT)), 1.0)
    return fdg
end

In [ ]:
function FatiguehistoryVariable(ψhPos_in,αPrev,αbPrev)
    α = ψhPos_in #(s_in ^2 + η)*ψhPos_in
    if α >= αPrev
       αb = αbPrev + abs(α-αPrev)
    else
       αb = αbPrev
    end
return αb
end

In [ ]:
function αNC(s_in,ψhPos_in)
    gs = (s_in^2) / (s_in^2 + m*(1-s_in)*(0.5+0.5*s_in))
    α = gs * ψhPos_in
return α
end

In [ ]:
order = 1
reffe_PF = ReferenceFE(lagrangian ,Float64,order)
V0_PF = TestFESpace(model ,reffe_PF;conformity =:H1)
U_PF = TrialFESpace(V0_PF)
sh = zero(V0_PF)

# Modified

In [ ]:
reffe_Disp = ReferenceFE(lagrangian ,VectorValue{2,Float64},order)
V0_Disp = TestFESpace(model,reffe_Disp;conformity =:H1,
    dirichlet_tags = ["DirichletLeftPoint","DirichletRightPoint","DirichletBottom","DirichletTop"],
    dirichlet_masks =[(true,false),(true,false), (false ,true), (false ,true)])

uh = zero(V0_Disp)

In [ ]:
degree = 2*order
Ω= Triangulation(model)
dΩ= Measure(Ω,degree)

In [ ]:
labels = get_face_labeling(model)
LoadTagId = get_tag_from_name(labels ,"DirichletTop")
Γ_Load = BoundaryTriangulation(model ,tags = LoadTagId)
dΓ_Load = Measure(Γ_Load ,degree)
n_Γ_Load = get_normal_vector(Γ_Load)

In [ ]:
nls = NLSolver(
  show_trace=true,
  method=:newton,
  linesearch=BackTracking(), iterations = 10)
solver = FESolver(nls)

In [ ]:
function crack_tip_tracker(sh,tol)
    damagetracker = Float64[]
    sVec = Float64[]
    coords = get_node_coordinates(Ω)
    
    sVec = get_free_dof_values(sh)
    
    for i in 1:length(sVec)
        if sVec[i] <= tol
            push!(damagetracker,i)
        end
    end
   damagetracker = round.(Int,damagetracker)
    xloc = Float64[]
    yloc = Float64[]
    if damagetracker!= Float64[]
        for i in 1:length(damagetracker)
            push!(xloc,coords[damagetracker[i]][1])
            push!(yloc,coords[damagetracker[i]][2])
        end
        Locx, index= findmax(xloc)
        Locy = yloc[index]
        return (Locx,Locy)
    else 
       return "No damage"
    end
end

In [ ]:
function stepPhaseField(x0,vApp,cache,ψPlusPrev_in)
     U_PF = TrialFESpace(V0_PF)
     res(s,ϕ) = ∫( (3/4)*Gc*ls*∇(ϕ)⋅ ∇(s) + (degDer∘(s))*ψPlusPrev_in*ϕ - ((3/8)*Gc/ls)*ϕ)*dΩ
     op = FEOperator(res,U_PF,V0_PF)
     sh_out = FEFunction(U_PF,x0)
     sh_out, cache = solve!(sh_out,solver,op,cache)
  return sh_out, get_free_dof_values(sh_out), cache
end

In [ ]:
function  stepDisp(uh_in,sh_in ,vApp)
    uApp1(x) = VectorValue(0.0,0.0)
    uApp2(x) = VectorValue(0.0,0.0)
    uApp3(x) = VectorValue(0.0,-vApp)
    uApp4(x) = VectorValue(0.0,vApp)
    U_Disp = TrialFESpace(V0_Disp ,[uApp1,uApp2,uApp3,uApp4])
    a_Disp(u,v) =∫( (ε(v)⊙(σfun∘(ε(u),ε(uh_in),sh_in)) ) )*dΩ
    b_Disp(v) = 0.0
    op_Disp = AffineFEOperator(a_Disp ,b_Disp ,U_Disp ,V0_Disp)
    uh_out = solve(op_Disp)
    return  uh_out
end
     

In [ ]:
vApp = 0
const innerMax = 10
const tol_lim = 1e-10
count = 0

const T = 1
const n = 200
const Tₚ = T/n
const n1 = 16 # loadsteps
const delv = 0.001
const tp = Tₚ/n1
t = 0.0
cycle = 0
const dc = 1/n1

Load = Float64[]
Displacement = Float64[]
time = Float64[]
AppDisplacement = Float64[]
αEnergy = Float64[]
αDegrad = Float64[]
NoofCycles = Float64[]
Xloccrack = Float64[]
Yloccrack = Float64[]


push!(Xloccrack,(L/2))
push!(Yloccrack,0.5*H)
push!(Load, 0.0)
push!(Displacement, 0.0)

push!(time, t)
push!(AppDisplacement, vApp)
push!(NoofCycles, cycle)

push!(αEnergy, 0.0)
push!(αDegrad, 1.0)

sPrev = CellState(1.0,dΩ)
sh = project(sPrev ,model ,dΩ,order)
x0_PF = ones(Float64,num_free_dofs(V0_PF))
cache_1 = nothing

αPrev = CellState(0.0,dΩ)
αbPrev = CellState(0.0,dΩ)

αhPlusPrev = project(αPrev,model,dΩ,order)
αbhPlusPrev = project(αbPrev,model,dΩ,order)

FdhPrev = FatigueDegrad∘(αbhPlusPrev)

ψPlusPrev = CellState(ψ_Crit,dΩ)

while  t .< T
    t = t + tp
    vAppC = abs(4*delv/Tₚ * abs((((t-Tₚ/4)%Tₚ)+Tₚ)%Tₚ - Tₚ/2) - delv)
    vApp = round(vAppC,digits = 6)
    count = count .+ 1
    cycle = cycle + dc
    FdhPrev = FatigueDegrad∘(αbhPlusPrev)
    
    print("\n Entering  displacemtent  step$count:", float(vApp))
        
    for  inner = 1: innerMax
        ψhPlusPrev = project(ψPlusPrev ,model ,dΩ,order)
        RelErr = abs(sum(∫( (3/4)*Gc*ls*∇(sh)⋅ ∇(sh) + (degDer∘(sh))*ψhPlusPrev*sh)*dΩ-∫( ((3/8)*Gc/ls)*sh)*dΩ))/abs(sum(∫( ((3/8)*Gc/ls)*sh)*dΩ))
        
        println("\n Relative error =", float(RelErr), "\n")
        sh,x0_PF,cache_1 = stepPhaseField(x0_PF,vApp,cache_1,ψhPlusPrev)
        uh = stepDisp(uh,sh,vApp)
        
        ψhPos_in =ψPos∘(ε(uh),FdhPrev)
        
        update_state!( new_EnergyState ,ψPlusPrev ,ψhPos_in)
        if   RelErr  < 1e-6
            break
        end
    end
    
    if  crack_tip_tracker(sh,tol_lim) =="No damage"
        push!(Xloccrack,0.5*L)
        push!(Yloccrack,0.5*H)
    else
        (X_t, Y_t) = crack_tip_tracker(sh,tol_lim)
        push!(Xloccrack,X_t)
        push!(Yloccrack,Y_t)
    end
    
    ψhPosOrg_in = ψPosOrg∘(ε(uh))
    αPrev = αNC∘(sh,ψhPosOrg_in)
    αbPrev = FatiguehistoryVariable∘(αPrev,αhPlusPrev,αbhPlusPrev)
    
    αhPlusPrev = project(αPrev,model,dΩ,order)
    αbhPlusPrev = project(αbPrev,model,dΩ,order)
    
    FdhPrev = FatigueDegrad∘(αbhPlusPrev)
    
    αbPrevVal = evaluate(αbPrev,VectorValue(0.5*L, 2*CH))
    Fdegrad = evaluate(FdhPrev,VectorValue(0.5*L, 2*CH))
    
    push!(time, t)
    push!(AppDisplacement, vApp)
    push!(NoofCycles, cycle)
    
    push!(αEnergy, αbPrevVal)
    push!(αDegrad, Fdegrad)
    
    Node_Force = sum(∫(n_Γ_Load⋅(σfun∘(ε(uh),ε(uh),sh)))*dΓ_Load)
    
    push!(Load , Node_Force[2])
    push!( Displacement , vApp)
    if mod(count,n1) == 0
        writevtk(Ω,"results_DENT_Test$count",cellfields= ["uh"=>uh,"s"=>sh , "epsi"=>ε(uh),"Fdeg"=> FdhPrev, "alphabar"=> αbhPlusPrev])
    end
 end     

In [ ]:
plot(Displacement,Load)

In [ ]:
plot(NoofCycles[1:930],Xloccrack[1:930].-(0.5*L))

In [ ]:
using DelimitedFiles
using CSV
using Plots

In [ ]:
using LaTeXStrings

In [ ]:
parent_path = "../PlottingFiles/"     # Go up one level and define the new folder name
mkpath(parent_path)

In [ ]:
cd("../PlottingFiles/")

In [ ]:
NoofCyclesCSV = writedlm("kf01_Step16NoofCyclesPhlsfullVolDev$ls.csv",  NoofCycles, ',')
XloccrackCSV = writedlm("kf01_Step16XloccrackPhlsfullVolDev$ls.csv",  Xloccrack, ',')
YloccrackCSV = writedlm("kf01_Step16YloccrackPhlsfullVolDev$ls.csv",  Yloccrack, ',')
LoadCSV = writedlm("kf01_Step16LoadPhlsfullVolDev$ls.csv",  Load, ',')